In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## added selenium code
from selenium import webdriver
import time
from bs4 import BeautifulSoup
## For parallel execution
from multiprocessing import Pool
from tqdm.auto import tqdm
from datetime import date

today = date.today().strftime('%Y-%m-%d')

In [2]:
#Scrape data from all pages
# Enable javascript and disable automation flag
options = webdriver.ChromeOptions()
options.add_argument("--enable-javascript")
options.add_argument("--dns-prefetch-disable");
options.add_argument("--disable-gpu");
options.add_argument("--disable-blink-features=AutomationControlled")
# start the instance
driver = webdriver.Chrome(options=options)

In [3]:
def Parser(url):
    """ Opens the URL passed to the function"""
    driver.get(url)
    return(driver)
    
def Get_Properties(pages):
    """Loops through all pages of listing, and saves their info into list."""
    all_links=[]
    for i in tqdm(range(1,pages)):
        Parser(f"https://www.kv.ee/?act=search.simple&page_size=100&deal_type=1&county=1&search_type=new&parish=1061&page=[{i}]")
        try:
            #accepts the cookies
            driver.find_element_by_id("onetrust-accept-btn-handler").click()
        except:
            pass
        finally:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            properties = soup.find_all("tr", class_="object-type-apartment object-item")
            for property in properties:
                links = property.find_all("a",{"class":"object-title-a text-truncate"})
                for link in links:
                    all_links.append(link['href'])
    return(all_links)

In [4]:
##Generate the property list
links = Get_Properties(35)

  0%|          | 0/34 [00:00<?, ?it/s]

In [5]:

all_info=[]
def Properties_info(i):
            d = {}
            Parser(links[i])
            try:
                driver.find_element_by_id("onetrust-accept-btn-handler").click()
            except:
                pass
            finally:
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                page_content = soup.find("div", {"class":"object-article-details"})
                price = page_content.find("div",{"class":"object-price"}).findChildren()[0]
                descr = soup.find("div",{"class":"object-article-body"}).find("p")
                aadress= soup.find('h1',{'class':'title'}).get_text(strip=True)
                #loop through rows in info table
                for items in page_content.find('table',{"class":"table-lined object-data-meta"}).find_all('tr'):
                    data = [item.get_text(strip=True) for item in items.find_all(['th','td'])]
                    d[data[0]]=data[-1]
                    d['url']=links[i]
                    d['hind']=price.get_text(strip=True)
                    d['aadress']=aadress
                    d['description'] = descr
                    d['date'] = today
            all_info.append(d)

In [6]:
for i in tqdm(range(0,len(links))):
    Properties_info(i)

  0%|          | 0/3400 [00:00<?, ?it/s]

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (Session info: chrome=106.0.5249.91)
Stacktrace:
0   chromedriver                        0x000000010bc3f788 chromedriver + 4515720
1   chromedriver                        0x000000010bbc39d3 chromedriver + 4008403
2   chromedriver                        0x000000010b85612a chromedriver + 413994
3   chromedriver                        0x000000010b842b7f chromedriver + 334719
4   chromedriver                        0x000000010b842301 chromedriver + 332545
5   chromedriver                        0x000000010b85d6a3 chromedriver + 444067
6   chromedriver                        0x000000010b8be028 chromedriver + 839720
7   chromedriver                        0x000000010b8aa603 chromedriver + 759299
8   chromedriver                        0x000000010b880990 chromedriver + 588176
9   chromedriver                        0x000000010b881a75 chromedriver + 592501
10  chromedriver                        0x000000010bc0

In [ ]:
len(all_info)

In [120]:
filename = "data/"+today+"-dataset.csv"
pd.DataFrame(all_info).to_csv(filename)